In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
! pip install -q transformers[sentencepiece] datasets evaluate


In [3]:
import pandas as pd
import transformers

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/Capstone3

/content/drive/MyDrive/Capstone3


In [6]:
model_name = "distilroberta-base"

In [7]:
df = pd.read_csv("imdb_movies.csv")
df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/
3,Lightyear,"$71,101,257",5.2,32K,"['Animation', 'Action', 'Adventure']",While spending years attempting to return home...,"['galaxy', 'spaceship', 'robot', 'rocket', 'sp...",Angus MacLane,"['Jason Headley', 'Matthew Aldrich', 'Chris Ev...",Angus MacLane,-2022,/title/tt10298810/
4,Spiderhead,not-released,5.4,23K,"['Action', 'Crime', 'Drama']","In the near future, convicts are offered the c...","['discover', 'medical', 'test', 'reality', 'fi...",Joseph Kosinski,"['Rhett Reese', 'Paul Wernick', 'Chris Hemswor...",George Saunders,-2022,/title/tt9783600/


In [8]:
df = df.dropna().reset_index(drop=True)

In [9]:
genre_list = []
indices_to_drop = []

for idx,Genres in enumerate(df.Generes.to_list()):
  generes_list = eval(Genres)

  if len(generes_list):
    generes_list.append(generes_list[0])
  else:
    indices_to_drop.append(idx)

df = df.drop(indices_to_drop).reset_index(drop=True)
df.shape

(23412, 12)

In [10]:
genre_count = df['Generes'].value_counts()
threshold = int(len(df) * 0.005)
rare_genres = [ cat for cat, count in genre_count.items() if count <threshold]
len(rare_genres), rare_genres[:5]

(686,
 ["['Action', 'Adventure', 'Thriller']",
  "['Crime', 'Mystery', 'Thriller']",
  "['Action', 'Sci-Fi', 'Thriller']",
  "['Action', 'Sci-Fi']",
  "['Horror', 'Sci-Fi', 'Thriller']"])

In [11]:
rare_indices_to_drop = [idx for idx, row in df.iterrows() if row['Generes'] in rare_genres]
len(rare_indices_to_drop)

9249

In [12]:
df = df.drop(rare_indices_to_drop).reset_index(drop=True)
df.shape

(14163, 12)

In [13]:
df['Generes'].value_counts

<bound method IndexOpsMixin.value_counts of 0                         ['Action', 'Drama']
1           ['Action', 'Adventure', 'Sci-Fi']
2                         ['Action', 'Drama']
3        ['Animation', 'Action', 'Adventure']
4                ['Action', 'Crime', 'Drama']
                         ...                 
14158                     ['Comedy', 'Crime']
14159                     ['Comedy', 'Crime']
14160                     ['Comedy', 'Crime']
14161                     ['Comedy', 'Crime']
14162                     ['Comedy', 'Crime']
Name: Generes, Length: 14163, dtype: object>

In [14]:
len(df['Generes'].value_counts())

47

In [15]:
labels = list(set(df.Generes.to_list()))
label_count = len(labels)
labels, label_count

(["['Drama', 'Mystery', 'Thriller']",
  "['Action', 'Crime', 'Thriller']",
  "['Adventure', 'Comedy', 'Drama']",
  "['Crime', 'Drama', 'Thriller']",
  "['Comedy', 'Drama']",
  "['Drama']",
  "['Action', 'Comedy']",
  "['Crime', 'Drama', 'Romance']",
  "['Comedy', 'Crime', 'Drama']",
  "['Biography', 'Drama']",
  "['Animation', 'Adventure', 'Comedy']",
  "['Drama', 'Horror', 'Mystery']",
  "['Western']",
  "['Adventure', 'Comedy', 'Family']",
  "['Drama', 'Horror', 'Thriller']",
  "['Comedy', 'Crime']",
  "['Horror', 'Mystery', 'Thriller']",
  "['Action', 'Drama', 'Romance']",
  "['Comedy']",
  "['Horror', 'Sci-Fi']",
  "['Action', 'Crime', 'Drama']",
  "['Crime', 'Drama']",
  "['Action']",
  "['Action', 'Comedy', 'Drama']",
  "['Crime', 'Thriller']",
  "['Action', 'Adventure', 'Drama']",
  "['Drama', 'Thriller']",
  "['Action', 'Adventure', 'Crime']",
  "['Crime', 'Drama', 'Mystery']",
  "['Horror']",
  "['Comedy', 'Drama', 'Romance']",
  "['Action', 'Drama', 'Thriller']",
  "['Adventu

In [16]:
df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/
3,Lightyear,"$71,101,257",5.2,32K,"['Animation', 'Action', 'Adventure']",While spending years attempting to return home...,"['galaxy', 'spaceship', 'robot', 'rocket', 'sp...",Angus MacLane,"['Jason Headley', 'Matthew Aldrich', 'Chris Ev...",Angus MacLane,-2022,/title/tt10298810/
4,Spiderhead,not-released,5.4,23K,"['Action', 'Crime', 'Drama']","In the near future, convicts are offered the c...","['discover', 'medical', 'test', 'reality', 'fi...",Joseph Kosinski,"['Rhett Reese', 'Paul Wernick', 'Chris Hemswor...",George Saunders,-2022,/title/tt9783600/


In [17]:
df.describe(include='object')

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
count,14163,14163,14163,14163,14163,14163,14163,14163,14163,14163,14163,14163
unique,13958,993,88,1542,47,14134,12767,7901,14127,10106,227,13958
top,Rage,not-released,no-rating,0,['Drama'],none,[],Woody Allen,"['See producer', 'See preliminary cast']",Woody Allen,-2022,/title/tt0114224/
freq,4,4558,556,556,895,18,828,33,18,33,756,4


In [18]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df)

In [19]:
ds

Dataset({
    features: ['movie title', 'Run Time', 'Rating', 'User Rating', 'Generes', 'Overview', 'Plot Kyeword', 'Director', 'Top 5 Casts', 'Writer', 'year', 'path'],
    num_rows: 14163
})

In [20]:
ds[0]

{'movie title': 'Top Gun: Maverick',
 'Run Time': '$170,000,000 (estimated)',
 'Rating': '8.6',
 'User Rating': '187K',
 'Generes': "['Action', 'Drama']",
 'Overview': "After more than thirty years of service as one of the Navy's top aviators, Pete Mitchell is where he belongs, pushing the envelope as a courageous test pilot and dodging the advancement in rank that would ground",
 'Plot Kyeword': "['fighter jet', 'sequel', 'u.s. navy', 'fighter aircraft', 'pilot', 'naval aviation', 'jet fighter', 'military pilot', 'aviation', 'aircraft']",
 'Director': 'Joseph Kosinski',
 'Top 5 Casts': "['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise', 'Jennifer Connelly', 'Miles Teller']",
 'Writer': 'Jim Cash',
 'year': '-2022',
 'path': '/title/tt1745960/'}

In [21]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [22]:
tokenizer.tokenize(ds[0]['Overview'][:150])

['After',
 'Ġmore',
 'Ġthan',
 'Ġthirty',
 'Ġyears',
 'Ġof',
 'Ġservice',
 'Ġas',
 'Ġone',
 'Ġof',
 'Ġthe',
 'ĠNavy',
 "'s",
 'Ġtop',
 'Ġav',
 'i',
 'ators',
 ',',
 'ĠPete',
 'ĠMitchell',
 'Ġis',
 'Ġwhere',
 'Ġhe',
 'Ġbelongs',
 ',',
 'Ġpushing',
 'Ġthe',
 'Ġenvelope',
 'Ġas',
 'Ġa',
 'Ġcourageous',
 'Ġt',
 'es']

In [23]:
tokenizer(ds[0]['Overview'][:150])

{'input_ids': [0, 4993, 55, 87, 16984, 107, 9, 544, 25, 65, 9, 5, 5027, 18, 299, 6402, 118, 3629, 6, 9141, 5275, 16, 147, 37, 12918, 6, 3784, 5, 21035, 25, 10, 24219, 326, 293, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
def tokenize_desc(x) :
  return tokenizer(x['Overview'], truncation=True, padding=True)

In [25]:
tokenized_ds = ds.map(tokenize_desc, batched=True)

Map:   0%|          | 0/14163 [00:00<?, ? examples/s]

In [26]:
tokenized_ds[0]

{'movie title': 'Top Gun: Maverick',
 'Run Time': '$170,000,000 (estimated)',
 'Rating': '8.6',
 'User Rating': '187K',
 'Generes': "['Action', 'Drama']",
 'Overview': "After more than thirty years of service as one of the Navy's top aviators, Pete Mitchell is where he belongs, pushing the envelope as a courageous test pilot and dodging the advancement in rank that would ground",
 'Plot Kyeword': "['fighter jet', 'sequel', 'u.s. navy', 'fighter aircraft', 'pilot', 'naval aviation', 'jet fighter', 'military pilot', 'aviation', 'aircraft']",
 'Director': 'Joseph Kosinski',
 'Top 5 Casts': "['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise', 'Jennifer Connelly', 'Miles Teller']",
 'Writer': 'Jim Cash',
 'year': '-2022',
 'path': '/title/tt1745960/',
 'input_ids': [0,
  4993,
  55,
  87,
  16984,
  107,
  9,
  544,
  25,
  65,
  9,
  5,
  5027,
  18,
  299,
  6402,
  118,
  3629,
  6,
  9141,
  5275,
  16,
  147,
  37,
  12918,
  6,
  3784,
  5,
  21035,
  25,
  10,
  24219,
  1296,
  4792,
  8

In [27]:
row = tokenized_ds[0]
row['Overview'], row['input_ids']

("After more than thirty years of service as one of the Navy's top aviators, Pete Mitchell is where he belongs, pushing the envelope as a courageous test pilot and dodging the advancement in rank that would ground",
 [0,
  4993,
  55,
  87,
  16984,
  107,
  9,
  544,
  25,
  65,
  9,
  5,
  5027,
  18,
  299,
  6402,
  118,
  3629,
  6,
  9141,
  5275,
  16,
  147,
  37,
  12918,
  6,
  3784,
  5,
  21035,
  25,
  10,
  24219,
  1296,
  4792,
  8,
  38990,
  5,
  19026,
  11,
  7938,
  14,
  74,
  1255,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1])

In [28]:
tokenizer.vocab['Ġmore']

55

In [29]:
labels

["['Drama', 'Mystery', 'Thriller']",
 "['Action', 'Crime', 'Thriller']",
 "['Adventure', 'Comedy', 'Drama']",
 "['Crime', 'Drama', 'Thriller']",
 "['Comedy', 'Drama']",
 "['Drama']",
 "['Action', 'Comedy']",
 "['Crime', 'Drama', 'Romance']",
 "['Comedy', 'Crime', 'Drama']",
 "['Biography', 'Drama']",
 "['Animation', 'Adventure', 'Comedy']",
 "['Drama', 'Horror', 'Mystery']",
 "['Western']",
 "['Adventure', 'Comedy', 'Family']",
 "['Drama', 'Horror', 'Thriller']",
 "['Comedy', 'Crime']",
 "['Horror', 'Mystery', 'Thriller']",
 "['Action', 'Drama', 'Romance']",
 "['Comedy']",
 "['Horror', 'Sci-Fi']",
 "['Action', 'Crime', 'Drama']",
 "['Crime', 'Drama']",
 "['Action']",
 "['Action', 'Comedy', 'Drama']",
 "['Crime', 'Thriller']",
 "['Action', 'Adventure', 'Drama']",
 "['Drama', 'Thriller']",
 "['Action', 'Adventure', 'Crime']",
 "['Crime', 'Drama', 'Mystery']",
 "['Horror']",
 "['Comedy', 'Drama', 'Romance']",
 "['Action', 'Drama', 'Thriller']",
 "['Adventure', 'Drama']",
 "['Action', 'Thr

In [30]:
labels.index( "['Comedy', 'Crime']")

15

In [31]:
def categorize(x):
  return {"labels": [labels.index(genre) for genre in x["Generes"]]}

In [32]:
categorized_ds = tokenized_ds.map(categorize, batched=True)
categorized_ds

Map:   0%|          | 0/14163 [00:00<?, ? examples/s]

Dataset({
    features: ['movie title', 'Run Time', 'Rating', 'User Rating', 'Generes', 'Overview', 'Plot Kyeword', 'Director', 'Top 5 Casts', 'Writer', 'year', 'path', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14163
})

In [33]:
categorized_ds[0]

{'movie title': 'Top Gun: Maverick',
 'Run Time': '$170,000,000 (estimated)',
 'Rating': '8.6',
 'User Rating': '187K',
 'Generes': "['Action', 'Drama']",
 'Overview': "After more than thirty years of service as one of the Navy's top aviators, Pete Mitchell is where he belongs, pushing the envelope as a courageous test pilot and dodging the advancement in rank that would ground",
 'Plot Kyeword': "['fighter jet', 'sequel', 'u.s. navy', 'fighter aircraft', 'pilot', 'naval aviation', 'jet fighter', 'military pilot', 'aviation', 'aircraft']",
 'Director': 'Joseph Kosinski',
 'Top 5 Casts': "['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise', 'Jennifer Connelly', 'Miles Teller']",
 'Writer': 'Jim Cash',
 'year': '-2022',
 'path': '/title/tt1745960/',
 'input_ids': [0,
  4993,
  55,
  87,
  16984,
  107,
  9,
  544,
  25,
  65,
  9,
  5,
  5027,
  18,
  299,
  6402,
  118,
  3629,
  6,
  9141,
  5275,
  16,
  147,
  37,
  12918,
  6,
  3784,
  5,
  21035,
  25,
  10,
  24219,
  1296,
  4792,
  8

In [34]:
row = categorized_ds[0]
row['labels']

36

In [35]:
split_ds = categorized_ds.train_test_split(0.1, seed=42)
split_ds

DatasetDict({
    train: Dataset({
        features: ['movie title', 'Run Time', 'Rating', 'User Rating', 'Generes', 'Overview', 'Plot Kyeword', 'Director', 'Top 5 Casts', 'Writer', 'year', 'path', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 12746
    })
    test: Dataset({
        features: ['movie title', 'Run Time', 'Rating', 'User Rating', 'Generes', 'Overview', 'Plot Kyeword', 'Director', 'Top 5 Casts', 'Writer', 'year', 'path', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1417
    })
})

# Modeling

In [36]:
! pip install transformers[torch]

In [37]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [38]:
bs = 32
epochs = 16
lr = 3.75e-4

TrainingArguments [Documents](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments://)

In [41]:
args = TrainingArguments(
    "models",
    learning_rate = lr,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    #fp16=True,
    evaluation_strategy='epoch',
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs,
    num_train_epochs=epochs,
    weight_decay=0.01,
    report_to='none'
)

In [43]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=label_count)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [44]:
import evaluate
import numpy as np
def accuracy(eval_preds):
  metric = evaluate.load("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [45]:
trainer = Trainer(
    model,
    args,
    train_dataset = split_ds['train'],
    eval_dataset = split_ds['test'],
    tokenizer = tokenizer,
    compute_metrics = accuracy
)

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.980657,0.170078


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-46-3435b262f1ae>", line 1, in <cell line: 1>
    trainer.train()
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1555, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1937, in _inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2271, in _maybe_log_save_evaluate
    metrics = self.evaluate(ignore_keys=ignore_keys_for_eval)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 3011, in evaluate
    output = eval_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 3304, in evaluation_l